<a href="https://colab.research.google.com/github/koliby777/pokus-cislo/blob/master/Nov%C3%BD_jazykov%C3%BD_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Nový jazykový model**

# Krok 1: Příprava prostředí Google Colab

In [ ]:
# Instalace PyTorch
!pip install torch torchvision
# Instalace transformerů
!pip install transformers
from transformers import AutoTokenizer
import torch
from torch.utils.data import DataLoader, RandomSampler
from transformers import BertForSequenceClassification, AdamW
from torch.nn import CrossEntropyLoss


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
#Připojení Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Krok 2: Tokenizace

In [ ]:
# Příklad použití tokenizéru

tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased') #  Model cased rozlišuje velká a malá písmena
"""
Pro specifické úkoly: Některé modely mohou být optimalizovány pro konkrétní úkoly,
jako je otázka a odpověď, sentimentální analýza atd. V takovém případě vyber model,
který nejlépe odpovídá...
"""

# Tokenizace textu
text = "Nechtějte vážiti lehce řeči mojí, z nebeť přichází věští duch; zákon nezbytný ve všem světě stojí a vše tu svůj zaplatí dluh."
# Tokenizace textu. Výsledek je slovník obsahující tokenizovaný text
# a pozornostní masku (attention mask), která určuje, na které tokeny se má model soustředit,
# maska říká modelu, aby se zaměřil pouze na relevantní tokeny a ignoroval padding.
# Tokenizujeme text voláním metody tokenizer. Argument return_tensors='pt' znamená,
# že chceme výstup ve formátu PyTorch tensorů. Výsledek je slovník obsahující tokeny (input_ids)
# a pozornostní masku (attention_mask).

encoded_input = tokenizer(text, return_tensors='pt')
print("Tokenizovaný vstup:", encoded_input)

# Dekódování tokenizovaného textu zpět na původní text.
# Předáváme tokeny (input_ids) z našeho zakódovaného vstupu.
decoded_text = tokenizer.decode(encoded_input['input_ids'][0])
print("\nDekódovaný text:", decoded_text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Tokenizovaný vstup: {'input_ids': tensor([[  101, 19416, 11640, 11636, 26694, 10112,   190, 10661, 98568, 10141,
         10237, 10419,   346, 10112, 15515, 46912, 14868,   117,   194, 10554,
         11044, 12774, 16475, 66883,   190, 11636, 54578, 22620, 10237,   132,
         89487, 10554, 10305, 11530, 65754, 10323,   190, 66714, 65043, 61177,
           169,   190, 14747, 13055, 42839, 10339, 48590, 18619, 63940, 18593,
           119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]])}

Dekódovaný text: [CLS] Nechtějte vážiti lehce řeči mojí, z nebeť přichází věští duch ; zákon nezbytný ve všem světě stojí a vše tu svůj 

# Krok 3: Příprava dat

In [ ]:
from torch.utils.data import Dataset

"""
Definice nové třídy CustomDataset, která dědí z PyTorch Dataset. Tato třída bude reprezentovat náš dataset:
"""
class CustomDataset(Dataset):
    """
    Konstruktor třídy. Přijímá seznam textů (texts), instanci tokenizéru (tokenizer) a
    volitelně maximální délku sekvence (max_length). Tyto parametry jsou uloženy jako atributy třídy:
    """
    def __init__(self, texts, tokenizer, max_length=512):
        self.tokenizer = tokenizer  # Tokenizér pro převod textů na tokeny. Uložení tokenizéru jako atribut třídy.
        self.texts = texts  # Seznam textů, které chceme tokenizovat.  Uložení seznamu textů jako atribut třídy.
        self.max_length = max_length  # Maximální délka sekvence po tokenizaci. Uložení maximální délky sekvence jako atribut třídy.
                                      # Toto omezení je důležité pro zajištění, že všechny sekvence mají stejnou délku,
                                      # což je potřeba pro zpracování v dávkách (batch processing).

    """
    Metoda, která vrací celkový počet textů v datasetu. Tato metoda je vyžadována PyTorch DataLoaderem pro iteraci přes dataset:
    """
    def __len__(self):
        return len(self.texts)  # Vrátí počet textů v datasetu

    """
    Metoda, která vrací jednu položku z datasetu na základě indexu (idx). V tomto případě se jedná o tokenizaci
     konkrétního textu. Metoda tokenizer převádí text na sekvenci tokenů s použitím volitelných argumentů
      jako max_length, truncation (zkrácení textu, pokud přesahuje maximální délku), padding
       ('max_length' znamená, že všechny sekvence budou doplněny na maximální délku) a return_tensors="pt"
        (vrátí tokeny jako PyTorch tensory). Nakonec metoda vrátí tokenizované input_ids a attention_mask pro daný text:
    """
    def __getitem__(self, idx):
        text = self.texts[idx]  # Získá text na indexu idx
        # Tokenizuje text s omezením na maximální délku a s použitím paddingu a truncation
        inputs = self.tokenizer(text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors="pt")
        # Vrátí tokenizované id a pozornostní masku jako dvojici
        return inputs.input_ids[0], inputs.attention_mask[0], label

# Příklad použití ????
texts = ["První text.", "Druhý text.", "osmdesaty text ", "Nechtějte vážiti lehce řeči mojí, z nebeť přichází věští duch; zákon nezbytný ve všem světě stojí a vše tu svůj zaplatí dluh."]
dataset = CustomDataset(texts, tokenizer)


# Krok 4: Vytvoření modelu

BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2): Načte model BERT, který byl předtrénován na mnohojazyčných datech. Parametr num_labels=2 indikuje, že model bude použit pro úlohu klasifikace s dvěma třídami (např. pozitivní/negativní).

AdamW(model.parameters(), lr=5e-5): Inicializuje AdamW optimizer, který se často používá pro trénování modelů založených na Transformeru. Parametr lr udává rychlost učení.

In [ ]:
# Předpokládáme, že `dataset` a `tokenizer` jsou již definovány
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Krok 5: Trénování modelu

Trénovací smyčka

In [ ]:

train_dataloader = DataLoader(dataset, sampler=RandomSampler(dataset), batch_size=8)

# Definice ztrátové funkce
loss_fn = CrossEntropyLoss()

model.train()

for epoch in range(4):  # Počet epoch
    for step, batch in enumerate(train_dataloader):
        b_input_ids, b_input_mask, b_labels = batch
        b_input_ids = b_input_ids.squeeze(1)  # Upravit rozměry, pokud je to potřeba

        model.zero_grad()

        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=None)
        logits = outputs.logits

        # Převod štítků na tensor a případně na GPU, pokud model běží na GPU
        b_labels = b_labels.to(logits.device)

        loss = loss_fn(logits, b_labels)

        loss.backward()
        optimizer.step()

        if step % 100 == 0:
            print(f"Epoch: {epoch}, Step: {step}, Loss: {loss.item()}")


NameError: name 'label' is not defined